In [21]:
from autogen_ext.models.ollama import OllamaChatCompletionClient
ollama_model_client = OllamaChatCompletionClient(
    model="llama3.2"
)

In [22]:
from dotenv import load_dotenv
import os
load_dotenv()

api_key = os.getenv('GOOGLE_API_KEY')

In [23]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

gemini_model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=api_key 
)

In [24]:
from autogen_agentchat.agents import AssistantAgent

agentX = AssistantAgent(
    "primary",
    model_client=ollama_model_client,
    system_message="You are a helpful AI assistant.",
)

agentY = AssistantAgent(
    "critic" , 
    model_client=gemini_model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

## Creating a Team

In [25]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

text_termination = TextMentionTermination("APPROVE")

team = RoundRobinGroupChat([agentX , agentY], termination_condition=text_termination)

## Running a Team

In [26]:
result = await team.run(task="Write a short poem about the fall season.")
print(result)
print(result.messages)

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 36, 45, 679793, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=105), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 36, 50, 286924, tzinfo=datetime.timezone.utc), content="As summer's warmth begins to fade,\nThe trees don vibrant hues displayed,\nGolden, crimson, and amber bright,\nA colorful canvas in the autumn light.\n\nThe air is crisp, the winds do blow,\nLeaves rustle, swirling to and fro,\nNature's final dance before winter's sleep,\nA fleeting beauty, secrets keep.\n\nThe scent of woodsmoke wafts through air,\nAs earthy smells and spices share,\nThe forest floor, a carpet deep,\nA crunchy blanket, soft and sweet to keep.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_token

In [27]:
await team.reset()  # Reset the team for a new task.
# This method will clear the team’s state, including all agents. 
# It will call the each agent’s on_reset() method to clear the agent’s state.

async for chunk in team.run_stream(task="Write a short poem about the fall season."):
    print(chunk)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 36, 52, 201454, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=70) metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 36, 55, 359931, tzinfo=datetime.timezone.utc) content="As summer's warmth begins to fade,\nGolden leaves in whispers shade,\nCrimson maples stand, ablaze with light,\nAutumn's palette paints the night.\n\nThe air is crisp, the wind is cold,\nNature's final dance, young and old,\nFleeting moments of beauty rare,\nA season's farewell, beyond compare." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=101, completion_tokens=3) metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 36, 55, 976370, tzinfo=datetime.timezone.utc) content='APPROVE\n' type='TextMessage'
messages=[TextMessage(source='user', models_usage=None, metada

## Observing a Team

In [28]:
from autogen_agentchat.base import TaskResult

# When the team stops, it returns a TaskResult object with all the messages produced by the agents in the team.

await team.reset()
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 36, 55, 992652, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=41, completion_tokens=103) metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 37, 0, 449556, tzinfo=datetime.timezone.utc) content="As summer's warmth begins to fade,\nThe earth awakens from its shade,\nGolden leaves and crimson hue,\nA fiery glow, both old and new.\n\nThe air is crisp, the winds do blow,\nAnd nature's final dance begins to show,\nThe trees stand tall, their branches bare,\nA fleeting beauty, beyond compare.\n\nThe scent of woodsmoke fills the air,\nAs earthy aromas waft with care,\nThe stars appear in darkening sky,\nAnd fall's sweet magic passes by." type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=137, completion_tokens=3) metadata={} created_at=datetime.datetime(2025, 7, 5, 8, 

In [29]:
import asyncio

async def work(x):
    await asyncio.sleep(x)
    return f"Finished after {x} seconds"

# Run two tasks concurrently
task1 = asyncio.create_task(work(2))
task2 = asyncio.create_task(work(3))

# Wait for both
result1 = await task1
result2 = await task2


In [30]:
from autogen_agentchat.ui import Console

await team.reset() 
await Console(team.run_stream(task="Write a short poem about the fall season.")) 

# team.run(...) is an awaitable coroutine.
# team.run_stream(...) returns an async generator.e.

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Amidst the trees, a colorful sight
Golden hues and crimson bright
The leaves fall slow, a gentle hue
As nature's final dance begins anew

The air is crisp, the winds do blow
A time for change, as seasons go
The earthy scent of fallen leaves
Invigorates the senses, as autumn eaves

The sun sets early, a fiery glow
Painting the sky with shades of low
A fleeting moment, pure and bright
As fall's sweet magic takes its flight.
---------- TextMessage (critic) ----------
APPROVE



TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 4, 115836, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=107), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 9, 4168, tzinfo=datetime.timezone.utc), content="Amidst the trees, a colorful sight\nGolden hues and crimson bright\nThe leaves fall slow, a gentle hue\nAs nature's final dance begins anew\n\nThe air is crisp, the winds do blow\nA time for change, as seasons go\nThe earthy scent of fallen leaves\nInvigorates the senses, as autumn eaves\n\nThe sun sets early, a fiery glow\nPainting the sky with shades of low\nA fleeting moment, pure and bright\nAs fall's sweet magic takes its flight.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=130, completion_t

## Stopping the Team 

In [31]:
from autogen_agentchat.conditions import ExternalTermination
import asyncio

external_termination = ExternalTermination()

team = RoundRobinGroupChat(
    [agentX, agentY],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
Crimson leaves and amber light,
Dance upon the autumn night.
Golden hues and scarlet bright,
A final flourish before winter's sight.

The air is cool, the winds do sway,
As nature's canvas ends its day.
The scent of earth and woodsmoke sweet,
Invigorates the senses to retreat.

The sun sets early, a burning ember,
Painting the sky with fiery splendor.
A fleeting moment, pure and free,
Autumn's beauty for you and me.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 10, 941917, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=166, completion_tokens=102), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 15, 931430, tzinfo=datetime.timezone.utc), content="Crimson leaves and amber light,\nDance upon the autumn night.\nGolden hues and scarlet bright,\nA final flourish before winter's sight.\n\nThe air is cool, the winds do sway,\nAs nature's canvas ends its day.\nThe scent of earth and woodsmoke sweet,\nInvigorates the senses to retreat.\n\nThe sun sets early, a burning ember,\nPainting the sky with fiery splendor.\nA fleeting moment, pure and free,\nAutumn's beauty for you and me.", type='TextMessage')], stop_reason='External termination requested')

In [32]:
# asyncio.create_task(Console(team.run_stream(...)))

# 1. "Run the stream"
# 2. "Pipe the output into Console()"
# 3. "Do it without waiting immediately — I’ll await it when I’m ready"
# 4 .Then [await run] lets you pause and wait for it to complete.

In [33]:
await team.reset()

task = asyncio.create_task(team.run(task="Write a short poem about the fall season."))
result = await task

print(result)
print(result.messages)


messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 15, 955803, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=112), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 20, 971444, tzinfo=datetime.timezone.utc), content="As autumn leaves begin to sway,\nGolden hues and crimson day,\nThe trees stand tall, their branches wide,\nA fiery spectacle, side by side.\n\nThe air is crisp, the wind is cold,\nNature's final dance, before winter's fold,\nThe scent of woodsmoke wafts through the air,\nAs earthy aromas fill the autumn lair.\n\nThe sun sets early, days grow short,\nFading light, a gentle, golden resort,\nThe stars appear, like diamonds bright,\nA night of rest, in autumn's silent night.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(p

In [34]:
# Use await directly:
# When you want the result right away
# No other tasks need to run in parallel

await team.reset()
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Waits till done before moving forward

# Use asyncio.create_task():
# When you want to start a task in the background
# Do other things before waiting
# Handle multiple tasks concurrently

await team.reset()
task = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))
# do something else...
await task  # Now wait for it



---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
As summer's warmth begins to fade,
The trees don crimson, golden shade,
Their leaves, a rustling whisper low,
A symphony of autumn's woe.

The air is crisp, the winds do blow,
And nature's final dance begins to show,
The forest floor, a carpet deep,
Of orange, red, and yellow sleep.

The scent of woodsmoke fills the air,
As earthy aromas waft with care,
A season's farewell, a story told,
Of autumn's beauty, young and old.
---------- TextMessage (critic) ----------
APPROVE

---------- TextMessage (user) ----------
Write a short poem about the fall season.
---------- TextMessage (primary) ----------
As autumn leaves begin to sway,
Golden hues and crimson stay,
A fleeting dance, a final spin,
Before the winter's chill within.

The air is crisp, the winds do blow,
Scents of woodsmoke start to flow,
Nature's canvas, painted bright,
A colorful farewell to the light.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 28, 340856, tzinfo=datetime.timezone.utc), content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=63), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 31, 88667, tzinfo=datetime.timezone.utc), content="As autumn leaves begin to sway,\nGolden hues and crimson stay,\nA fleeting dance, a final spin,\nBefore the winter's chill within.\n\nThe air is crisp, the winds do blow,\nScents of woodsmoke start to flow,\nNature's canvas, painted bright,\nA colorful farewell to the light.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=96, completion_tokens=3), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 37, 31, 631836, tzinfo=datetime.timezone.utc), content='APPROVE\n', type='TextMessage')], stop_

## Resuming in a Team

In [35]:
await Console(team.run_stream())# Resume the team to continue the last task.


---------- TextMessage (primary) ----------
I'm glad you liked it! If you have any other requests or need help with anything else, feel free to ask!
---------- TextMessage (critic) ----------
I'm ready for your next request!  Let's see what you'd like to do.

---------- TextMessage (primary) ----------
I'd love to create a short story for you. Do you have any specific prompts or themes in mind, or would you like me to come up with something entirely random?

Also, is there a particular tone or style you'd like the story to convey? For example, would you like it to be:

* Light-hearted and humorous
* Dark and mysterious
* Adventurous and action-packed
* Heartwarming and sentimental

Let me know, and I'll get started!
---------- TextMessage (critic) ----------
I'd like a short story, approximately 500 words, that is **light-hearted and humorous**.  The story should involve **animals** and their interactions with **humanity**, focusing on a relatable and slightly absurd situation.  Think 

TaskResult(messages=[TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=117, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 55, 22, 354373, tzinfo=datetime.timezone.utc), content="I'm glad you liked it! If you have any other requests or need help with anything else, feel free to ask!", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=125, completion_tokens=23), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 55, 24, 553847, tzinfo=datetime.timezone.utc), content="I'm ready for your next request!  Let's see what you'd like to do.\n", type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=171, completion_tokens=98), metadata={}, created_at=datetime.datetime(2025, 7, 5, 8, 55, 29, 135320, tzinfo=datetime.timezone.utc), content="I'd love to create a short story for you. Do you have any specific prompts or themes in mind, or would you like me to come up w

## Aborting a Team

In [40]:
from autogen_core import CancellationToken
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
await team.reset()
run = asyncio.create_task(
    team.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

cancellation_token.cancel()
# It is used to programmatically cancel the ongoing team.run(...) task.

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError:
    print("Task was cancelled.")

# Different from stopping a team, aborting a team will immediately stop the team 
# and raise a CancelledError exception.

Task was cancelled.


In [48]:
cancellation_token2 = CancellationToken()

await team.reset()

run = asyncio.create_task(
    team.run(
        task="Hey how are you",
        cancellation_token=cancellation_token2,
    )
)

result = await run
print(result)

# if you use the same cancellation token then it will not work and give error.

messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 3, 4, 227296, tzinfo=datetime.timezone.utc), content='Hey how are you', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=65), metadata={}, created_at=datetime.datetime(2025, 7, 5, 9, 3, 7, 126687, tzinfo=datetime.timezone.utc), content="I'm doing well, thank you for asking! I'm a large language model, so I don't have emotions or feelings in the way that humans do, but I'm always happy to chat and help with any questions or topics you'd like to discuss. How about you? How's your day going so far?", type='TextMessage')] stop_reason='The group chat is stopped.'


In [52]:
import asyncio
from autogen_core import CancellationToken

async def run_and_cancel_after_delay():

    cancellation_token = CancellationToken()

    await team.reset()

    # Start running the team with run_stream
    run_stream = team.run_stream(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )

    # Create an async task to iterate messages from run_stream
    async def consume_stream():
        async for message in run_stream:
            print("Message:", message)

    consumer_task = asyncio.create_task(consume_stream())

    # Wait for 3 seconds before cancelling
    await asyncio.sleep(3)

    print("Cancelling the task now...")
    cancellation_token.cancel()

    try:
        await consumer_task
    except asyncio.CancelledError:
        print("Task was cancelled.")

# Run the async function
await run_and_cancel_after_delay()


I am here
Message: source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 5, 9, 6, 13, 44610, tzinfo=datetime.timezone.utc) content='Translate the poem to Spanish.' type='TextMessage'


Error processing publish message for primary_6565807a-cd73-468c-8584-621438174dae/6565807a-cd73-468c-8584-621438174dae
Traceback (most recent call last):
  File "/Users/akashjain/Desktop/Autogen/AutoGenv/lib/python3.12/site-packages/autogen_core/_single_threaded_agent_runtime.py", line 605, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/akashjain/Desktop/Autogen/AutoGenv/lib/python3.12/site-packages/autogen_core/_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/akashjain/Desktop/Autogen/AutoGenv/lib/python3.12/site-packages/autogen_agentchat/teams/_group_chat/_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/akashjain/Desktop/Autogen/AutoGenv/lib/python3.12/site-packages/autogen_core/_routed_age

Cancelling the task now...
Task was cancelled.
